In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summary/news_summary_more.csv
/kaggle/input/news-summary/news_summary.csv
/kaggle/input/wikihow-summarization/wikihowAll.csv


In [50]:
!pip install contractions

In [51]:
data = pd.read_csv('../input/wikihow-summarization/wikihowAll.csv', delimiter=',')


In [52]:
import numpy as np  
import pandas as pd 

import re
import nltk

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint,Callback

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [53]:
data = data.dropna()

In [54]:
!pip install pyspellchecker

In [55]:
import nltk
from nltk.data import find
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize.treebank import TreebankWordDetokenizer
import contractions
import unicodedata
from bs4 import BeautifulSoup
import emoji
import re
from spellchecker import SpellChecker
from typing import List

class Preprocessor:

    resources = [
        'tokenizers/punkt',
        'corpora/stopwords',
        'corpora/wordnet',
        'taggers/averaged_perceptron_tagger'
    ]

    def __init__(self) -> None:
        for resource in self.resources:
            try:
                find(resource)
                print(f"{resource} is already downloaded.")
            except LookupError:
                print(f"{resource} not found. Downloading...")
                download(resource.split('/')[1])

        # Stopword removal
        self.stop_words = set(stopwords.words('english'))
        # Initialize the WordNet lemmatizer
        self.lemmatizer = WordNetLemmatizer()

    def lower_sentence(self, sentence: str) -> str:
        '''
        Lowercase the sentence.
        :param data: The sentence to lowercase.
        :return: The lowercased sentence
        :rtype: str
        '''
        return sentence.lower()

    def remove_emails(self, sentence: str) -> str:
        '''
        Remove emails from the sentence.
        :param sentence: The sentence to remove emails from.
        :type sentence: str
        :return: The sentence without emails.
        :rtype: str
        '''
        return re.sub(r"\S*@\S*\s?", "", sentence)

    def remove_nonascii_diacritic(self, sentence: str) -> str:
        '''

        Remove diacritics from the sentence.

        :param sentence: The sentence to remove diacritics from.

        :type sentence: str

        :return: The sentence without diacritics.

        :rtype: str
        '''

        return unicodedata.normalize("NFKD", sentence).encode("ascii", "ignore").decode("utf-8", "ignore")

    def clean_html(self, sentence: str) -> str:
        '''
        Remove HTML tags from the sentence.
        :param sentence: The sentence to remove HTML tags from.
        :type sentence: str
        :return: The sentence without HTML tags.
        :rtype: str
        '''
        return BeautifulSoup(sentence, "html.parser").get_text()

    def replace_repeated_chars(self, sentence: str) -> str:
        '''
        Replace repeated characters in the sentence.
        :param sentence: The sentence to replace repeated characters in.
        :type sentence: str
        :return: The sentence with replaced repeated characters.
        :rtype: str
        '''
        # Replace consecutive occurrences of ',', '!', '.', and '?' with a single occurrence
        return re.sub(r'([,!?.])\1+', r'\1', sentence)

    def translate_emojis_to_text(self, sentence: str) -> str:
        '''
        Translate emojis in the sentence to text.
        :param sentence: The sentence to translate emojis to text.
        :type sentence: str
        :return: The sentence with translated emojis to text.
        :rtype: str
        '''
        line = ''
        for char in sentence:
            if emoji.is_emoji(char):
                emoji_text = emoji.demojize(char)[1:-1].replace('_', ' ')
                line += emoji_text
            else:
                line += char

        return line

    def expand_sentence(self, sentence: str) -> str:
        '''
        Expand the contractions in the sentence.
        :param sentence: The sentence to expand contractions in.
        :type sentence: str
        :return: The sentence with expanded contractions.
        :rtype: str
        '''
        return contractions.fix(sentence)

    def remove_url(self, sentence: str) -> str:
        '''
        Remove URLs from the sentence.
        :param sentence: The sentence to remove URLs from.
        :type sentence: str
        :return: The sentence without URLs.
        :rtype: str
        '''
        return re.sub("((http\://|https\://|ftp\://)|(www.))+(([a-zA-Z0-9\.-]+\.[a-zA-Z]{2,4})|([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}))(/[a-zA-Z0-9%:/-_\?\.'~]*)?", '', sentence)

    def remove_possessives(self, sentence: str) -> str:
        '''
        Strip possessives from the sentence.
        :param sentence: The sentence to strip possessives from.
        :type sentence: str
        :return: The sentence without possessives.
        :rtype: str
        '''
        # Stripping the possessives
        sentence = sentence.replace("'s", '')
        sentence = sentence.replace('’s', '')
        sentence = sentence.replace('s’', 's')
        sentence = sentence.replace("s'", 's')
        return sentence

    def remove_extra_space(self, sentence: str) -> str:
        '''
        Remove extra spaces from the sentence.
        :param sentence: The sentence to remove extra spaces from.
        :type sentence: str
        :return: The sentence without extra spaces.
        :rtype: str
        '''
        return re.sub(r'\s+', ' ', sentence).strip()


    def check_sentence_spelling(self, sentence: List[str]) -> List[str]:
        '''
        Check the spelling of the words in the sentence.
        :param sentence: The sentence to check the spelling of.
        :type sentence: list
        :return: The sentence with corrected spelling.
        :rtype: list
        '''
        spell = SpellChecker()
        corrected_sentence = []
        for word in sentence:
            if word != '':
                correction = spell.correction(word)
                if correction is not None:
                    corrected_sentence.append(correction)
                else:
                    corrected_sentence.append(word)
            else:
                corrected_sentence.append('')
        return corrected_sentence

    def tokenize_sentence(self, sentence: str) -> List[str]:
        '''
        Tokenize the sentence.
        :param sentence: The sentence to tokenize.
        :type sentence: str
        :return: The tokenized sentence.
        :rtype: str
        '''
        return nltk.word_tokenize(sentence)


    def remove_stop_words(self, sentence: List[str]) -> List[str]:
        '''
        Remove stop words from the sentence.
        :param sentence: The sentence to remove stop words from.
        :type sentence: list[str]
        :return: The sentence without stop words.
        :rtype: list[str]
        '''
        return [word for word in sentence if word not in self.stop_words]

    def lemm_sentence(self, sentence: List[str]) -> List[str]:
        '''
        Lemmatize the sentence.
        :param sentence: The sentence to lemmatize.
        :type sentence: list[str]
        :return: The lemmatized sentence.
        :rtype: list[str]
        '''
        # Perform POS tagging
        pos_tags = pos_tag(sentence)
        # Lemmatize each word based on its POS tag
        lemmatized_words = []
        for word, pos in pos_tags:
            # Map Penn Treebank POS tags to WordNet POS tags
            if pos.startswith('N'):  # Nouns
                pos = 'n'
            elif pos.startswith('V'):  # Verbs
                pos = 'v'
            elif pos.startswith('J'):  # Adjectives
                pos = 'a'
            elif pos.startswith('R'):  # Adverbs
                pos = 'r'
            else:
                pos = 'n'  # Default to noun if POS tag not found

            # Lemmatize the word using the appropriate POS tag
            lemma = self.lemmatizer.lemmatize(word, pos=pos)
            lemmatized_words.append(lemma)
        return lemmatized_words

    def detokenize_sentence(self, sentence: List[str]) -> str:
        '''
        Detokenize the sentence.
        :param sentence: The sentence to detokenize.
        :type sentence: list[str]
        :return: The detokenized sentence.
        :rtype: str
        '''
        return TreebankWordDetokenizer().detokenize(sentence)

    def remove_emojis(self,text:str) -> str:
        '''
        Removes specific patterns like (😃,🚀) and emojis from the given text.
        :type text: list[str]
        :return: Text without emojis.
        :rtype: str
        '''
        emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002500-\U00002BEF"  # chinese char
                u"\U00002700-\U000027BF"  # Dingbats
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u"\U00010000-\U0010ffff"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u200d"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)

        return text

    def remove_emoticons(self,text:str) -> str:
        '''
        Removes specific patterns like[:) | :(] and emoticons from the given text.
        :type text: list[str]
        :return: Text without emoticons.
        :rtype: str
        '''
        # Define a regular expression pattern to match emoticons
        emoticon_pattern = re.compile(r':(\)+)|:-(\))+|;(\))+|:-(D)+|:(D)+|;-(D)+|x(D)+|X(D)+|:-(\()+|:(\()+|:-(/)+|:(/)+|:-(\))+||:(\))+||:-(O)+|:(O)+|:-(\*)+|:(\*)+|<(3)+|:(P)+|:-(P)+|;(P)+|;-(P)+|:(S)+|>:(O)+|8(\))+|B-(\))+|O:(\))+', flags=re.IGNORECASE)
        # Remove emoticons using the pattern
        return emoticon_pattern.sub('', text)

    def remove_non_alphabetic(self,text:str) -> str:
        '''
        Removes non-alphabetic characters from the given text.
        :type text: str
        :return: Text without non-alphabetic characters.
        :rtype: str
        '''
        cleaned_text = re.sub(r'\W+', ' ', text)
        return cleaned_text

    def clean(self, line: str, steps: List[str] = None, empty: str ='Normal') -> List[str]:
        '''
        Clean the line and return it as a list of tokens
        :param line: the line to clean
        :type line: str
        :param steps: list of steps to apply
        :type steps: list[str]
        :return: the cleaned line as a list of tokens
        :rtype: list
        '''
        # Default steps to apply if none are specified
        default_steps = [
            'translate_emojis_to_text',
            'lower_sentence',
            'remove_nonascii_diacritic',
            'remove_emails',
            'clean_html',
            'remove_url',
            'replace_repeated_chars',
            'expand_sentence',
            'remove_possessives',
            'remove_extra_space',
            'tokenize_sentence',
            'check_sentence_spelling',
            'remove_stop_words',
            'lemm_sentence'
        ]

        # Use specified steps if provided, otherwise use default steps
        if steps is None:
            steps = default_steps

        # Define the processing functions
        processing_functions = {
            'translate_emojis_to_text': self.translate_emojis_to_text,
            'lower_sentence': self.lower_sentence,
            'remove_nonascii_diacritic': self.remove_nonascii_diacritic,
            'remove_emails': self.remove_emails,
            'clean_html': self.clean_html,
            'remove_url': self.remove_url,
            'replace_repeated_chars': self.replace_repeated_chars,
            'expand_sentence': self.expand_sentence,
            'remove_possessives': self.remove_possessives,
            'remove_extra_space': self.remove_extra_space,
            'tokenize_sentence': self.tokenize_sentence,
            'check_sentence_spelling': self.check_sentence_spelling,
            'remove_stop_words': self.remove_stop_words,
            'lemm_sentence': self.lemm_sentence,
            'detokenize_sentence': self.detokenize_sentence,
            'remove_emojis': self.remove_emojis,
            'remove_emoticons': self.remove_emoticons,
            'remove_non_alphabetic': self.remove_non_alphabetic
        }

        # Apply the specified steps
        for step in steps:
            if step in processing_functions:
                line = processing_functions[step](line)

        # Ensure tokenize_sentence was applied
        if isinstance(line, str):
            line = [line]

        if len(line) == 0:
            return [empty]

        return line
     



In [56]:
preprocess=Preprocessor()

tokenizers/punkt is already downloaded.
corpora/stopwords is already downloaded.
corpora/wordnet is already downloaded.
taggers/averaged_perceptron_tagger is already downloaded.


In [57]:

def clean_text_wiki(text):

    clean_text=preprocess.clean(text,
            ["lower_sentence","remove_emojis","remove_emoticons","remove_nonascii_diacritic",
            "remove_emails","clean_html",
            "remove_url","replace_repeated_chars","expand_sentence",
            "remove_extra_space","tokenize_sentence","remove_stop_words","detokenize_sentence"]
            ,"")[0]
    return clean_text

cleaning_text = []

for t in data['text']:
    cleaning_text.append(clean_text_wiki(t))

In [58]:
def clean_summary_wiki(text):

    clean_text=preprocess.clean(text,
            ["lower_sentence","remove_emojis","remove_emoticons","remove_nonascii_diacritic",
            "remove_emails","clean_html",
            "remove_url","replace_repeated_chars","expand_sentence",
            "remove_extra_space"]
            ,"")[0]
    return clean_text


cleaning_summary = []
for t in data['headline']:
    cleaning_summary.append(clean_summary_wiki(t))



#### save cleaning text

In [59]:
data['cleaning_text'] = cleaning_text
data['cleaning_summary'] = cleaning_summary
data['cleaning_summary']=data['cleaning_summary'].replace('', np.nan)
data['cleaning_text']=data['cleaning_text'].replace('', np.nan)

data=data.dropna(axis = 0)

In [60]:
len(data)

214293

In [61]:
for i in range(5):
    print("text: ",data['cleaning_text'][i])
    print("summary: ",data['cleaning_summary'][i])
    print("\n")

text:  photographer,keep necessary lens,cords,batteries quadrant home studio . paints kept brushes,cleaner,canvas,print supplies ink,etc . make broader groups areas supplies make finding easier,limiting search much smaller area . ideas include:essential supplies area--things use every day . inspiration reference area . dedicated work area . infrequent secondary supplies area,tucked way .;,mean cleaning entire studio,means keeping area immediately around desk,easel,pottery wheel,etc . clean night . discard trash unnecessary materials wipe dirty surfaces . endeavor leave workspace way sit next day start working immediately,without work tidying . even rest studio bit disorganized,organized workspace help get business every time want make art .,visual people,lot artist clutter comes desire keep track supplies visually instead tucked sight . using jars,old glasses,vases,cheap,clear plastic drawers,keep things sight without leaving strewn haphazardly . ideas,beyond mentioned,include:canvas s

## Graph for show len of text and summary

In [62]:
text_counter = []
summary_counter = []

In [63]:
for sentance in data['cleaning_text']:
    text_counter.append(len(sentance.split()))
for sentance in data['cleaning_summary']:
    summary_counter.append(len(sentance.split()))

In [64]:
print(max(text_counter))
print((summary_counter[0]))

6759
84


In [65]:
# # plotting the data
# plt.figure(figsize=(14, 7))

# # plotting the distribution of text lengths
# plt.subplot(1, 2, 1)
# sns.histplot(text_counter, kde=True, color='blue', bins=150)
# plt.title('Distribution of Text Sentence Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')

# # plotting the distribution of summary lengths
# plt.subplot(1, 2, 2)
# sns.histplot(summary_counter, kde=True, color='green', bins=150)
# plt.title('Distribution of Summary Sentence Lengths')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')

# plt.tight_layout()
# plt.show()


In [66]:
count=0
for row in data['cleaning_summary']:
    if(len(row.split())<=30):
        count=count+1
print(count/len(data['cleaning_summary']))

0.34163038456692474


In [67]:
count=0
for row in data['cleaning_text']:
    if(len(row.split())<=150):
        count=count+1
print(count/len(data['cleaning_text']))

0.4959331382732987


# Select the Summaries and Text between max len defined above

In [68]:
limit_text_len = 70
limit_summary_len = 20

In [69]:
## delete all lenght larger than limitations

cleaned_text =np.array(data['cleaning_text'])
cleaned_summary=np.array(data['cleaning_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=limit_summary_len and len(cleaned_text[i].split())<=limit_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
data=pd.DataFrame({'cleaning_text':short_text,'cleaning_summary':short_summary})

In [70]:
data['cleaning_summary'] = data['cleaning_summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [71]:
data['cleaning_summary']

0       sostok tie up your horse., groom your horse. e...
1       sostok introduce yourself to his friends., tal...
2       sostok learn what he likes., ask him what he t...
3       sostok once again, if he says no, just shake i...
4       sostok eye contact., physical contact., jealou...
                              ...                        
8243      sostok determine how you think of books. eostok
8244    sostok shelve all of your books alphabetically...
8245    sostok place the books on the shelf according ...
8246    sostok this technique can also be a sub-techni...
8247    sostok use chrome to plate metals subject to c...
Name: cleaning_summary, Length: 8248, dtype: object

## split data 

In [72]:
from sklearn.model_selection import train_test_split

## split train , test and val 
x_tr, x_temp, y_tr, y_temp = train_test_split(np.array(data['cleaning_text']), np.array(data['cleaning_summary']),
                                              test_size=0.2, random_state=0, shuffle=True)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=0, shuffle=True)


In [73]:
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [74]:
therold_count = 2

count = 0
total_count = 0
frequnecy = 0
total_freq = 0

for key,counter in x_tokenizer.word_counts.items():
    total_count = total_count + 1
    total_freq = total_freq + counter
    if(counter < therold_count):
        count = count + 1
        frequnecy = frequnecy + counter
    
print("percentage of rare words in vocabulary:", (count/total_count) * 100)
print("total coverage of rare words in data:", (frequnecy/total_freq) * 100)

percentage of rare words in vocabulary: 40.04414163707897
total coverage of rare words in data: 3.22236893923599


In [75]:
x_tokenizer = Tokenizer(num_words = total_count-count) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences 
x_train_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

# padding zero upto maximum length
x_train_pad  =  pad_sequences(x_train_seq,  maxlen = limit_text_len, padding = 'post')
x_val_pad   =  pad_sequences(x_val_seq, maxlen = limit_text_len, padding = 'post')

# size of vocabulary ( +1 for padding token)
x_vocabulary   =  x_tokenizer.num_words + 1

print("size of vocabulary in X = {}".format(x_vocabulary))

size of vocabulary in X = 12497


In [76]:
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [77]:
therold_count = 2

count = 0
total_count = 0
frequnecy = 0
total_freq = 0

for key,counter in y_tokenizer.word_counts.items():
    total_count = total_count + 1
    total_freq = total_freq + counter
    if(counter < therold_count):
        count = count + 1
        frequnecy = frequnecy + counter
    
print("percentage of rare words in vocabulary:", (count/total_count) * 100)
print("total coverage of rare words in data:", (frequnecy/total_freq) * 100)

percentage of rare words in vocabulary: 47.7267943590823
total coverage of rare words in data: 4.451140512739979


In [78]:
y_tokenizer = Tokenizer(num_words = total_count - count) 
y_tokenizer.fit_on_texts(list(y_tr))

# convert text sequences into integer sequences 
y_train_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

# padding zero upto maximum length
y_train_pad    =   pad_sequences(y_train_seq, maxlen = limit_summary_len, padding = 'post')
y_val_pad   =   pad_sequences(y_val_seq, maxlen = limit_summary_len, padding = 'post')

# size of vocabulary ( +1 for padding token)
y_vocabulary  =   y_tokenizer.num_words + 1
print("Size of vocabulary in Y = {}".format(y_vocabulary))

Size of vocabulary in Y = 4968


In [79]:
!git clone "https://github.com/thushv89/attention_keras"

fatal: destination path 'attention_keras' already exists and is not an empty directory.


In [93]:
from tensorflow.keras.models import Model
from attention_keras.src.layers.attention import AttentionLayer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, TimeDistributed, Bidirectional, Masking

hidden_dim = 300
embedding_dim = 150
#### encoder

# Encoder input shape
Encoder_Input = Input(shape=(limit_text_len, ), name='Encoder_Input')
masked_input = Masking(mask_value=0)(Encoder_Input)

# Embedding layer
Encoder_Embedding =  Embedding(x_vocabulary, embedding_dim, trainable=True, name='Encoder_Embedding')(masked_input)
# Bidirectional Encoder LSTM 1
encoder_lstm1 = Bidirectional(LSTM(hidden_dim, return_sequences=True, return_state=True, dropout=0.5, recurrent_dropout=0.5), name='Bidirectional_Encoder_LSTM1')
enc_output_1, forward_h1, forward_c1, backward_h1, backward_c1 = encoder_lstm1(Encoder_Embedding)

# Concatenate forward and backward states
state_h1 = Concatenate()([forward_h1, backward_h1])
state_c1 = Concatenate()([forward_c1, backward_c1])

# Bidirectional Encoder LSTM 2
encoder_lstm2 = Bidirectional(LSTM(hidden_dim, return_sequences=True, return_state=True, dropout=0.5, recurrent_dropout=0.5), name='Bidirectional_Encoder_LSTM2')
enc_output_2, forward_h2, forward_c2, backward_h2, backward_c2 = encoder_lstm2(enc_output_1)

# Concatenate forward and backward states
state_h2 = Concatenate()([forward_h2, backward_h2])
state_c2 = Concatenate()([forward_c2, backward_c2])

# Bidirectional Encoder LSTM 3
encoder_lstm3 = Bidirectional(LSTM(hidden_dim, return_state=True, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), name='Bidirectional_Encoder_LSTM3')
enc_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm3(enc_output_2)

# Concatenate forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

############## decoder
# decoder input
dec_inputs = Input(shape=(None,), name='Decoder_Input')
masked_dec_inputs = Masking(mask_value=0)(dec_inputs)

# Embedding layer
dec_emb_layer = Embedding(y_vocabulary, embedding_dim, trainable=True, name='Decoder_Embedding')
dec_emb = dec_emb_layer(masked_dec_inputs)

decoder_lstm = LSTM(2*hidden_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2, name='Decoder_LSTM')
dec_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([enc_outputs, dec_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([dec_outputs, attn_out])

# Dense layer
decoder_dense = TimeDistributed(Dense(y_vocabulary, activation='softmax'), name='Final_Output_Layer')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([Encoder_Input, dec_inputs], decoder_outputs, name='Seq2Seq_Attention_Model')

# Print model summary
model.summary()

Model: "Seq2Seq_Attention_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      [(None, 70)]         0                                            
__________________________________________________________________________________________________
masking_4 (Masking)             (None, 70)           0           Encoder_Input[0][0]              
__________________________________________________________________________________________________
Encoder_Embedding (Embedding)   (None, 70, 150)      1874550     masking_4[0][0]                  
__________________________________________________________________________________________________
Bidirectional_Encoder_LSTM1 (Bi [(None, 70, 600), (N 1082400     Encoder_Embedding[0][0]          
____________________________________________________________________________

In [94]:
model.compile(optimizer = 'rmsprop',
              loss = 'sparse_categorical_crossentropy')

es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   verbose = 1,
                   patience = 1)

mc = ModelCheckpoint('ALL_best_model.h5',
                    monitor = 'val_loss',
                    mode = 'min',
                    verbose = 1,
                    save_best_only = True)

In [95]:
history = model.fit([x_train_pad, y_train_pad[:,:-1]], y_train_pad.reshape(y_train_pad.shape[0], y_train_pad.shape[1], 1)[:,1:],
                    epochs = 50,
                    callbacks = [es, mc],
                    batch_size = 264,
                    validation_data = ([x_val_pad,y_val_pad[:,:-1]], y_val_pad.reshape(y_val_pad.shape[0], y_val_pad.shape[1], 1)[:,1:]))

Train on 6598 samples, validate on 825 samples
Epoch 1/50
6336/6598 [===========================>..] - ETA: 1s - loss: 5.4683
Epoch 00001: val_loss improved from inf to 4.45451, saving model to ALL_best_model.h5
6598/6598 [==============================] - 41s 6ms/sample - loss: 5.4391 - val_loss: 4.4545
Epoch 2/50
6336/6598 [===========================>..] - ETA: 0s - loss: 4.5983
Epoch 00002: val_loss improved from 4.45451 to 4.28541, saving model to ALL_best_model.h5
6598/6598 [==============================] - 21s 3ms/sample - loss: 4.6003 - val_loss: 4.2854
Epoch 3/50
6336/6598 [===========================>..] - ETA: 0s - loss: 4.4524
Epoch 00003: val_loss improved from 4.28541 to 4.13040, saving model to ALL_best_model.h5
6598/6598 [==============================] - 21s 3ms/sample - loss: 4.4552 - val_loss: 4.1304
Epoch 4/50
6336/6598 [===========================>..] - ETA: 0s - loss: 4.3016
Epoch 00004: val_loss improved from 4.13040 to 4.04114, saving model to ALL_best_model.h5

In [96]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

# encode the input sequence to get the feature vector
encoder_model = Model(inputs=Encoder_Input, outputs=[enc_outputs, state_h, state_c])

# decoder setup
# below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(2*hidden_dim,))
decoder_state_input_c = Input(shape=(2*hidden_dim,))
decoder_hid_s_input = Input(shape=(limit_text_len, 2*hidden_dim))

# get the embeddings of the decoder sequence
decoder_emb2 = dec_emb_layer(dec_inputs) 
# to predict the next word in the sequence, set the initial states to the states from the previous time step
dec_outputs, state_h2, state_c2 = decoder_lstm(decoder_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])
attn_layer = AttentionLayer(name='attention_layer')
#attention inference
attn_out, attn_states_inf = attn_layer([decoder_hid_s_input, dec_outputs])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([dec_outputs, attn_out])

# a dense softmax layer to generate prob dist. over the target vocabulary
dec_outputs = decoder_dense(decoder_inf_concat) 

# final decoder model
decoder_model = Model(
    [dec_inputs] + [decoder_hid_s_input,decoder_state_input_h, decoder_state_input_c],
    [dec_outputs] + [state_h2, state_c2])


In [97]:
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

# Save tokenizers
import pickle

with open('x_tokenizer.pkl', 'wb') as f:
    pickle.dump(x_tokenizer, f)

with open('y_tokenizer.pkl', 'wb') as f:
    pickle.dump(y_tokenizer, f)

In [98]:
def out_sequence(input_by_seq):
    # encode the input as state vectors.
    encoder_out, encoder_h, encoder_c = encoder_model.predict(input_by_seq)
    
    # generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # ppulate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop = False
    out_sentence = ''
    while not stop:
      
        output_tokens, Enc_h, Enc_c = decoder_model.predict([target_seq] + [encoder_out, encoder_h, encoder_c])


        # get token
        token_index = np.argmax(output_tokens[0, -1, :])
        token = reverse_target_word_index[token_index]
        
        if(token!='eostok'):
            out_sentence += ' '+token

        # exit condition: either hit max length or find stop word.
        if (token == 'eostok'  or len(out_sentence.split()) >= (limit_summary_len-1)):
            stop = True

        # update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = token_index

        # Update internal states
        encoder_h, encoder_c= Enc_h,Enc_c

    return out_sentence

In [99]:
def seq2summary(input_seq):
    out = ''
    for i in input_seq:
        if((i != 0 and i != target_word_index['sostok']) and i != target_word_index['eostok']):
            out = out + reverse_target_word_index[i] + ' '
    return out

def seq2text(input_seq):
    out = ''
    for i in input_seq:
        if(i != 0):
            out = out + reverse_source_word_index[i]+' '
    return out

In [100]:
for i in range(0, 10):
    print("Review:", seq2text(x_train_pad[i]))
    print("Original summary:", seq2summary(y_train_pad[i]))
    print("Predicted summary:", out_sequence(x_train_pad[i].reshape(1, limit_text_len)))
    print("\n")

Review: use conditioner like long scent mixes well honey stir conditioner honey bowl thoroughly combined store extra conditioner old conditioner bottle later use use ratio honey conditioner make larger batch shampoo hair use honey conditioner would normal conditioner distribute small amount hair rinse finished leave conditioner hair 5 10 minutes finish shower greater lightening effect hair feels sticky rinsing decrease amount honey use increase amount conditioner 
Original summary: mix 1 4 cup honey and 1 2 cup conditioner use the conditioner after every shampoo 
Predicted summary:  make the spice and solution make the water add the water


Review: things improved greatly decorating still unusual fashion prepared carry 
Original summary: attach earring findings to each stopper through the stopper hole wear with 
Predicted summary:  use the ingredients of the ingredients


Review: rice ball purchased supermarket 100 g mushroom found forged throughout autumn select specific ingredients w

In [101]:
for i in range(0, 10):
    print("Review:", x_test[i])
    print("Original summary:", y_test[i])
    x_tr_seq2    =   x_tokenizer.texts_to_sequences(np.array([x_test[i]])) 
    x_tr2    =   pad_sequences(x_tr_seq2,  maxlen=limit_text_len, padding='post')
    print("Predicted summary:", out_sequence(x_tr2[0].reshape(1,limit_text_len)))
    print("\n")

Review: probably going hurt,try hard put balls feet toward ground stay flexible .,know land feet,slightly bend knees .,tuck ball first contact ground . land,land balls feet roll . lessen impact great deal.
Original summary: sostok prepare to feel some pain., bend your knees., use your knees and hips as shock absorbers. eostok
Predicted summary:  make a suitable of the top of the top


Review: someone clothes fire,wrap fire blanket .,use edges blanket protect hands prevent getting burned . roll blanket securely place.,instruct person danger stop,drop,roll . classic safety technique used diminish fire . person stop moving,drops ground,rolls fire suffocates.,burns caused fire evaluated medical professional soon possible . even think burns look minor,injury caused fire evaluated medical professional . take person whose clothes fire er immediately.
Original summary: sostok wrap someone whose clothes are burning in the fire blanket., have the person stop, drop, and roll., seek medical assist

In [90]:
# print("Review:",seq2text(x_tr2[0]))
# # print("Original summary:",seq2summary(y_tr[i]))
# print("Predicted summary:",decode_sequence(x_tr2[0].reshape(1,max_text_len)))
# print("\n")

In [105]:
from tensorflow.keras.models import load_model
import pickle
from attention_keras.src.layers.attention import AttentionLayer
custom_objects = {'AttentionLayer': AttentionLayer}
# Load models
encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5',custom_objects=custom_objects)

# Load tokenizers
with open('x_tokenizer.pkl', 'rb') as f:
    x_tokenizer = pickle.load(f)

with open('y_tokenizer.pkl', 'rb') as f:
    y_tokenizer = pickle.load(f)